In [4]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

In [24]:
base_url = 'https://www.avito.ru' 
url = 'https://www.avito.ru/sankt-peterburg/bytovaya_tehnika/dlya_kuhni/mikrovolnovye_pechi-ASgBAgICAkRglk_MB5pP?cd=1&pmax=4000&s=104&p={}'
ua = UserAgent()
headers = {'User-Agent': str(ua.chrome)}

In [17]:
r = requests.get(url.format(1), headers=headers)
r.status_code

200

In [20]:
for page in range(1, 10):
    r = requests.get(url.format(page), headers=headers)
    soup = BeautifulSoup(r.text, features='lxml')
    a_tags = soup.find_all('a', {'class':'snippet-link'})
    print(page, ':', len(a_tags))


1 : 56
2 : 56
3 : 56
4 : 56
5 : 56
6 : 56
7 : 56
8 : 56
9 : 56


In [23]:
a_tags[0].get('href')

'/sankt-peterburg/bytovaya_tehnika/dostavka_b.u._svch_pechey_s_garantiey_1405795017'

In [21]:
def get_last_page(soup):
    next_button = soup.find('span', {'data-marker':'pagination-button/next'})
    last_page = next_button.find_previous_sibling('span')
    return int(last_page.text)

In [22]:
get_last_page(soup)

24

In [32]:
def get_links(page):
    r = requests.get(url.format(page), headers=headers)
    if not r.ok:
        return []
    soup = BeautifulSoup(r.text, features='lxml')
    links = soup.find_all('a', {'class':'snippet-link'})
    if links:
        links = [base_url + link.get('href') for link in links]
    return links

In [31]:
len(get_links(20))

56